In [29]:
import numpy as np
import pandas as pd
import re
import string
import pickle

In [30]:
def remove_punctuation(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [31]:
with open('../static/model/model.pickle', 'rb') as f:
    model = pickle.load(f)

In [32]:
with open('../static/model/corpora/stopwords/english', 'r') as file:
    sw = file.read().splitlines()

In [33]:
vocab = pd.read_csv('../static/model/vocabulary.txt', header=None)
token = vocab[0].tolist()

In [34]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [35]:
def preprocessing(text):
    data = pd.DataFrame([text], columns=['tweet'])
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(x.lower() for x in x.split()))
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) for x in x.split()))
    data["tweet"] = data["tweet"].apply(remove_punctuation)
    data["tweet"] = data["tweet"].str.replace(r'\d+', '', regex=True)
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))
    return data["tweet"]

In [36]:
def vectorizer(ds, vocabulary):
    vectorized_lst = []

    for sentence in ds:
        sentence_lst = np.zeros(len(vocabulary))

        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_lst[i] = 1

        vectorized_lst.append(sentence_lst)

    vectorized_lst_new = np.asarray(vectorized_lst, dtype=np.float32)

    return vectorized_lst_new

In [44]:
def get_prediction(vectorized_txt):
    prediction = model.predict(vectorized_txt)
    if prediction == 1:
        return 'negative'
    else:
        return 'positive'

In [45]:
txt = "bad product. I hate it"
preprocessed_txt = preprocessing(txt)
vectorized_txt = vectorizer(preprocessed_txt, token)
prediction = get_prediction(vectorized_txt)
prediction

'negative'

In [46]:
txt = "good great awesome"
preprocessed_txt = preprocessing(txt)
vectorized_txt = vectorizer(preprocessed_txt, token)
prediction = get_prediction(vectorized_txt)
prediction

'positive'

In [47]:
txt = "awsome product. i love it"
preprocessed_txt = preprocessing(txt)
vectorized_txt = vectorizer(preprocessed_txt, token)
prediction = get_prediction(vectorized_txt)
prediction

'positive'